In [ ]:
%pip install pandas gspread pymysql python-dotenv matplotlib seaborn

### 1) 외부 데이터 수집

##### pymysql: MySQL -> Local
✅ 테이블 이름: products
|id|name|price|created_at|updated_at|rank_id|
|--|----|-----|----------|----------|-------|
|int|varchar(255)|int|timestamp|timestamp|int|

##### gspread: Google Spread -> Local
✅ 시트 이름: click_events
|customer_id|product_id|os|
|-----------|----------|--|
|uuid|int|'aos' \| 'ios'|


### 1-1. MySQL

In [ ]:
import pymysql
import os
import dotenv
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# point1: dotenv
dotenv.load_dotenv(os.path.join("key", ".env"))
DB_HOST = os.environ.get("DB_HOST")
DB_PORT = os.environ.get("DB_PORT")
DB_USER = os.environ.get("DB_USER")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
DB_SCHEMA = os.environ.get("DB_SCHEMA")

In [ ]:
# quiz1: 오늘 날짜를 YYYY-MM-DD 형식의 문자열로 뽑아내기
date_start = datetime.now() - timedelta(days=100)
date_start_str = date_start.strftime("%Y-%m-%d")
date_end = datetime.now() - timedelta(days=1)
date_end_str = date_end.strftime("%Y-%m-%d")
date_start_str, date_end_str

In [ ]:
# point2: pymysql
conn = pymysql.connect(host=DB_HOST, port=int(DB_PORT), user=DB_USER, password=DB_PASSWORD, db=DB_SCHEMA, charset='utf8')
with conn.cursor() as cursor:
    sql = "SELECT id, price, rank_id FROM products WHERE created_at >= %s AND created_at < %s AND rank_id IS NOT NULL"
    cursor.execute(sql, (date_start_str, date_end_str))
    rows = cursor.fetchall()
conn.close()

In [ ]:
df_products = pd.DataFrame(rows)
df_products.columns = ["product_id", "price", "rank_id"]
df_products

### 1-2. Google Sheet API

In [ ]:
import gspread

In [ ]:
GSPREAD_KEY_FILE = os.path.join("key", os.environ.get("GSPREAD_KEY_FILE"))
GSPREAD_FOLDER_ID = os.environ.get("GSPREAD_FOLDER_ID")

gc = gspread.service_account(filename=GSPREAD_KEY_FILE)

In [ ]:
ss = gc.open("click_events", folder_id=GSPREAD_FOLDER_ID)
wks = ss.get_worksheet(0)
data = wks.get_all_values()
columns = data[0]
columns[3] = "datetime"
body = map(
    lambda x: [x[0], int(x[1]), x[2], datetime.fromtimestamp(float(x[3]))], data[1:]
)
df_clicks = pd.DataFrame(body)
df_clicks.columns = columns
df_clicks

### 2) 테이블 분석

In [ ]:
df_products

In [ ]:
df_clicks

In [ ]:
df = df_clicks.merge(df_products, how="left", on="product_id")
df

In [ ]:
# rank_id, os별 click 개수
series_analysis = df.groupby(['rank_id', 'os']).count()['product_id']
series_analysis.name = "nclicks"
df_analysis = series_analysis.reset_index()
df_analysis

### 3) 데이터 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# price, rank 상관 관계
X = df['price']
Y = df['rank_id']
plt.scatter(X, Y)
plt.xlabel('Price')
plt.ylabel('Rank ID')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='rank_id', y='nclicks', hue='os', data=df_analysis)
plt.title('#Clicks by Rank ID and OS')
plt.xlabel('Rank ID')
plt.ylabel('#Clicks')
plt.legend(title='OS')
plt.show()

In [ ]:
# 피벗 테이블 생성
pivot_table = df_analysis.pivot(index='rank_id', columns='os', values='nclicks')

# 히트맵 그리기
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Heatmap of #Clicks by Rank ID and OS')
plt.xlabel('OS')
plt.ylabel('Rank ID')
plt.show()